In [112]:
from __future__ import print_function
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import dendrogram, linkage
import torch
import torch.nn as nn

ImportError: dlopen(/Users/orion/anaconda3/lib/python3.7/site-packages/torch/_C.cpython-37m-darwin.so, 9): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/orion/anaconda3/lib/python3.7/site-packages/torch/lib/libshm.dylib
  Reason: image not found

In [93]:
df = pd.read_csv('featurized_condensed.csv')

In [94]:
df = df.drop(columns=['Unnamed: 0', 'Ingredients'])

In [96]:
df = df.set_index('ID')

In [98]:
cuisines = list(set(df['Cuisine']))

In [99]:
binary_cuisines = {}
counter = 0
for i in cuisines:
    length = np.zeros(len(cuisines))
    length[counter] = 1
    length = length.tolist()
    counter += 1
    binary_cuisines[i] = length

In [101]:
lists = {}
for i in df.iterrows():
    i[1][0] = binary_cuisines[i[1][0]]
    lists[i[0]] = i[1][0]+i[1][1:].get_values().tolist()

In [108]:
class Net(nn.Module):
    def __init__(self, rep_size, hidden_size):
        super(Net, self).__init__()
        # add code

    def forward(self, x):
        x = x.view(-1,nobj+nrel) # reshape as size [B x (nobj+nrel) Tensor] if B=1
        # add code
        return output, hidden

NameError: name 'nn' is not defined